In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine

# --- CONFIGURAÇÃO DA CONEXÃO ---
# Use as mesmas credenciais do seu arquivo .env
DB_USER = "trading_user"
DB_PASS = "admin123"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "trading_db"

DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Cria o motor de conexão com o banco
engine = create_engine(DATABASE_URL)

print("Conexão com o banco de dados estabelecida com sucesso!")

Conexão com o banco de dados estabelecida com sucesso!


In [2]:
def load_backtest_data(backtest_id, db_engine):
    """
    Carrega todos os dados relevantes de um backtest do banco de dados.
    """
    # Carrega informações gerais do backtest
    backtest_info_df = pd.read_sql(f"SELECT * FROM backtests WHERE id = {backtest_id}", db_engine)
    if backtest_info_df.empty:
        raise ValueError(f"Backtest com ID {backtest_id} não encontrado.")
    backtest_info = backtest_info_df.iloc[0]

    # Carrega as métricas
    metrics_df = pd.read_sql(f"SELECT * FROM metrics WHERE backtest_id = {backtest_id}", db_engine)
    
    # Carrega os trades
    trades_df = pd.read_sql(f"SELECT * FROM trades WHERE backtest_id = {backtest_id}", db_engine)

    # Carrega os dados de preço do ativo para o período do backtest
    ticker = backtest_info['ticker']
    start_date = backtest_info['start_date']
    end_date = backtest_info['end_date']
    
    price_query = f"""
        SELECT date, open, high, low, close, volume 
        FROM prices p JOIN symbols s ON p.symbol_id = s.id
        WHERE s.ticker = '{ticker}' AND p.date BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY p.date
    """
    prices_df = pd.read_sql(price_query, db_engine, index_col='date', parse_dates=['date'])
    
    return {
        "info": backtest_info,
        "metrics": metrics_df,
        "trades": trades_df,
        "prices": prices_df
    }

In [6]:
# --- ESCOLHA O ID DO BACKTEST PARA ANALISAR ---
BACKTEST_ID_PARA_ANALISAR = 9 # Troque pelo ID que deseja ver

try:
    data = load_backtest_data(BACKTEST_ID_PARA_ANALISAR, engine)
    print("--- INFORMAÇÕES DO BACKTEST ---")
    print(data['info'])
    print("\n--- MÉTRICAS ---")
    print(data['metrics'].transpose())
    print("\n--- PRIMEIROS 5 TRADES ---")
    print(data['trades'].head())
except ValueError as e:
    print(e)

--- INFORMAÇÕES DO BACKTEST ---
id                                                          9
created_at                                         2025-09-29
ticker                                               MGLU3.SA
start_date                                         2021-01-01
end_date                                           2024-12-31
strategy_type                                       sma_cross
strategy_params_json    {'fast_length': 5, 'slow_length': 20}
initial_cash                                         100000.0
commission                                                0.0
status                                              COMPLETED
Name: 0, dtype: object

--- MÉTRICAS ---
                                   0
id                          8.000000
backtest_id                 9.000000
total_return_percentage     1.900913
sharpe_ratio                0.010620
max_drawdown_percentage     4.973475
win_rate_percentage        62.500000
total_net_pnl            1900.913352
total_close

In [7]:
prices_df = data['prices']
trades_df = data['trades']

# Cria a figura
fig = go.Figure()

# 1. Adiciona a linha de preço de fechamento
fig.add_trace(go.Scatter(
    x=prices_df.index,
    y=prices_df['close'],
    mode='lines',
    name='Preço de Fechamento',
    line=dict(color='blue')
))

# 2. Adiciona os marcadores de COMPRA
fig.add_trace(go.Scatter(
    x=trades_df['date_entry'],
    y=trades_df['price_entry'],
    mode='markers',
    name='Compra',
    marker=dict(color='green', symbol='triangle-up', size=10)
))

# 3. Adiciona os marcadores de VENDA
fig.add_trace(go.Scatter(
    x=trades_df['date_exit'],
    y=trades_df['price_exit'],
    mode='markers',
    name='Venda',
    marker=dict(color='red', symbol='triangle-down', size=10)
))

# Configura o layout
fig.update_layout(
    title=f"Backtest para {data['info']['ticker']} (Estratégia: {data['info']['strategy_type']})",
    xaxis_title='Data',
    yaxis_title='Preço',
    legend_title='Legenda',
    template='plotly_white'
)

fig.show()

In [7]:
# --- CÉLULA DE DEPURAÇÃO ---

trades_df = data['trades']

if trades_df.empty:
    print("✅ CONFIRMADO: O DataFrame de trades está VAZIO.")
    print("Nenhuma operação foi executada neste backtest, por isso não há marcadores no gráfico.")
else:
    print("⚠️ INTERESSANTE: O DataFrame de trades NÃO está vazio. Conteúdo:")
    print(trades_df)

✅ CONFIRMADO: O DataFrame de trades está VAZIO.
Nenhuma operação foi executada neste backtest, por isso não há marcadores no gráfico.
